In [9]:
import pandas as pd
import json
from datetime import datetime, timedelta
import os

root = "eye_trackers"


# print("Generating heatmap")
input_file = f"my_gaze_data_bea"

os.system(
    f"python ../gazeProcess.py {input_file}.csv {input_file}_clean.csv 1920 1080"
)


0

In [10]:
df = pd.read_csv(f'{root}/{input_file}_clean.csv')

with open('answers.json', 'r') as file:
    json_data = json.load(file)

json_obj = json_data[0]

initial_date = datetime.strptime(json_obj["initialDate"], "%Y-%m-%dT%H:%M:%S.%fZ")
df['current_time'] = pd.to_datetime(df['current_time'], format="%Y-%m-%dT%H:%M:%S.%fZ")

# Step 1: Create a new DataFrame with the rows where `time_difference` is less than zero
last_time_seconds = df[df['current_time'] - initial_date < timedelta(seconds=0)]['time_seconds'].iloc[-1]

# Step 2: Remove all rows where `time_seconds` is less than 0
df['time_seconds'] = df['time_seconds'] - last_time_seconds
df = df[df['time_seconds'] >= 0]

# Reset the index
df.reset_index(drop=True, inplace=True)

# Initialize the postID column
df['postID'] = None

# Iterate through each JSON object to assign postID
for obj in json_data:
    post_start_time = obj["PostStartTime"]
    post_end_time = obj["PostEndTime"]
    post_id = obj["postID"]
    
    # Assign `postID` based on `PostStartTime` and `PostEndTime`
    df.loc[(df['time_seconds'] >= post_start_time) & (df['time_seconds'] <= post_end_time), 'postID'] = post_id

# Remove all rows where `postID` is None
df = df[df['postID'].notna()]

In [11]:
unique_post_ids = df['postID'].unique()

for post_id in unique_post_ids:
    post_id_df = df[df['postID'] == post_id].drop(columns=['postID', 'current_time'])
    post_id_df['y'] =  post_id_df['y'] - 50
    post_id_df = post_id_df[['x', 'y'] + [col for col in post_id_df.columns if col not in ['x', 'y']]]
    file_name = f'{root}/my_gaze_bea_{int(post_id)}.csv'
    post_id_df.to_csv(file_name, index=False)
    print(f"Created file: {file_name}")

Created file: eye_trackers/my_gaze_bea_4.csv
Created file: eye_trackers/my_gaze_bea_7.csv
Created file: eye_trackers/my_gaze_bea_1.csv
Created file: eye_trackers/my_gaze_bea_6.csv
Created file: eye_trackers/my_gaze_bea_2.csv
Created file: eye_trackers/my_gaze_bea_5.csv


In [12]:
import os

# run generate.py
run = [1, 2, 4, 6, 7]

width = 1920
height = 1080
base = "images/"

# print("Generating heatmap")
input_file = f"{root}/my_gaze_bea_"
for image in run:

    os.system(
    f"python ../gazeheatplot.py {input_file}{image}.csv {width} {height} -b {base}{image}.png -o images/heatmap_{image}.png"
    )
